In [1]:
import pandas as pd
import os
import numpy as np
import re

In [2]:
pip install erddapy

Note: you may need to restart the kernel to use updated packages.


In [3]:
from erddapy import ERDDAP

In [4]:
IDlist = [
            'FORCE_Mar2018_ADCP_Currents',
    'FORCE_Mar2018_ADCP_Waves'
   #'force_3b1e_d478_b537' 3rd Dataset whhich is not found in cioos erdapp 
]

e = ERDDAP(server="https://dev.cioosatlantic.ca/erddap",
                 protocol="tabledap", )
e.auth = ("cioosatlantic", "4oceans")
e.response = "csv"

In [5]:
datasetdict = {}
unitdict = {}

In [6]:
for val in IDlist:
    e.dataset_id = val
    e.constraints = { "time<=": "2018-03-30T20:17:33Z",
                    }
    df = e.to_pandas()
    for col in df.columns:      
        if len(df[col].unique()) == 1:          
            df.drop([col], axis=1, inplace=True) 
    
    variables = list(df.columns)
    for i in range(len(variables)):
        variables[i] =  re.sub(r'\(.*?\) *', '', variables[i])
        
    variablelist = list(df.columns)
    newdict = dict(zip(variables, variablelist))
    unitdict.update(newdict)
    
    for i in range(len(variables)):
        datasetdict[variables[i]] = val
del unitdict['time ']          

In [7]:
def getdata(varname):
    e = ERDDAP(server="https://dev.cioosatlantic.ca/erddap",
                 protocol="tabledap", )
    e.auth = ("cioosatlantic", "4oceans")
    e.response = "csv"
    e.dataset_id = datasetdict.get(varname)
    e.variables = ['time', varname]
    e.constraints = {"time<=": "2018-05-23T17:47:30Z", "time>=": '2018-03-30T20:02:33Z'}
    df = e.to_pandas()
    df = df.sort_values(by=['time (UTC)'])
    df['time (UTC)'] = pd.to_datetime(df['time (UTC)'], format="%Y-%m-%dT%H:%M:%SZ")
    df = df.set_index(df['time (UTC)'].astype(np.datetime64))
    if varname == 'eastward_sea_water_velocity ' or 'upward_sea_water_velocity ' or 'northward_sea_water_velocity ':
        df = df[df[unitdict[varname]] !=0]
    if varname == 'sea_surface_wave_maximum_period ':
        df = df[df[unitdict[varname]] !=-9]
    return df

In [ ]:
import datetime as dt
from django.urls import include, re_path
import hvplot.pandas
import panel as pn
from holoviews import opts
import datashader as ds
import holoviews.operation.datashader as hd
from bokeh.models.formatters import DatetimeTickFormatter
import holoviews as hv